In [167]:
import time
import pandas as pd
import pyautogui

# Libs do WebScraping
import requests
from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [168]:
df = pd.read_table("csv/ChCh-Miner_durgbank-chem-chem.tsv", names=['drug_id1', 'drug_id2'])

In [169]:
df

,drug_id1,drug_id2
0,DB00862,DB00966
1,DB00575,DB00806
2,DB01242,DB08893
3,DB01151,DB08883
4,DB01235,DB01275
...,...,...
48509,DB00542,DB01354
48510,DB00476,DB01239
48511,DB00621,DB01120
48512,DB00808,DB01356


In [170]:
df = df.iloc[:1]

In [171]:
def get_interaction_result(drug_id1, drug_name_2):
    url = f"https://go.drugbank.com/drugs/{drug_id1}#taxonomy/drug-interactions"

    chrome_driver_path = '/home/edudev/Documents/dados_abertos/drugsbank/web_driver/chromedriver'

    # Configurar as opções do Chrome para executar em modo headless
    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")  # Maximizar a janela

    # Configurar o serviço do ChromeDriver
    service = Service(chrome_driver_path)

    # Criar a instância do WebDriver do Chrome
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)

    try:
        pyautogui.scroll(-30, x=2000, y=170)  # Rolar 3 "clics" para baixo (ajuste conforme necessário)

        # Esperar até que o campo de pesquisa esteja presente na página
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Search"]'))
        )

        search_input.clear()
        search_input.send_keys(drug_name_2)
        search_input.send_keys(Keys.RETURN)

        # Esperar até que a tabela de interações seja carregada
        time.sleep(5)  # Aguardar 5 segundos para garantir que a tabela tenha carregado completamente

        interactions_table = driver.find_element(By.ID, "drug-interactions-table")
        result_row = interactions_table.find_element(By.CSS_SELECTOR, "tbody > tr")
        result_text = result_row.find_element(By.XPATH, "./td[2]").text
        return result_text
    except:
        return "N/A"
    finally:
        driver.quit()

In [172]:
def get_drug_name(drug_id):
    url = f"https://go.drugbank.com/drugs/{drug_id}#taxonomy"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    drug_name = soup.select_one("h1")
    if drug_name:
        return drug_name.get_text(strip=True)
    return "N/A"

In [173]:
# Aplicar a função ao DataFrame e criar novas colunas
df["drug_name_1"] = df["drug_id1"].apply(get_drug_name)
df["drug_name_2"] = df["drug_id2"].apply(get_drug_name)
df["interaction_result"] = df.apply(lambda row: get_interaction_result(row["drug_id1"], row["drug_name_2"]), axis=1)

/tmp/ipykernel_57182/321433373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["drug_name_1"] = df["drug_id1"].apply(get_drug_name)
/tmp/ipykernel_57182/321433373.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["drug_name_2"] = df["drug_id2"].apply(get_drug_name)
/tmp/ipykernel_57182/321433373.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [174]:
df.head()

,drug_id1,drug_id2,drug_name_1,drug_name_2,interaction_result
0,DB00862,DB00966,Vardenafil,Telmisartan,Vardenafil may increase the antihypertensive a...
